### Load Structure

In [1]:
#1
# Load one survey instance
import json
import pandas as pd
import re


with open("data_raw/SurveyFormStructureElements_28316.json", "r", encoding="utf-8") as f:
    survey_structure = json.load(f)

# Sanity check: see what top-level keys and sample elements look like
print("Top-level keys:", list(survey_structure[0].keys()))
print("\nSurveyFormElements sample (first 1):")
print(survey_structure[0]["data"]["SurveyFormElements"][:1])
print("\nSurveyFormQuestionAnswers sample (first 3):")
print(survey_structure[0]["data"]["SurveyFormQuestionAnswers"][:3])

Top-level keys: ['manifest', 'schema', 'data']

SurveyFormElements sample (first 1):
[{'uuid': 'ac69a3a5-db77-4cee-af8f-1dfc8975cf32', 'fields': {'SurveyFormID': 28316, 'ElementID': 397320, 'ElementTypeID': 'Q ', 'ObjectName': None, 'Text': "<p>Please note that the questionnaire is not as long as it seems - multiple questions will be hidden based on your scenario.</p><p>Please read the guidelines below before filling in the questionnaire.\xa0<br><br>1. Read the questions carefully before answering to understand what exactly it is asking you;<br>2. Your feedback should describe in detail how you felt during the visit/interaction with the Advisor;<br>3. The Advisor's performance is the focus of this questionnaire, not prices, products, or store location;<br>4. Please share constructive feedback on what can be controlled by the Advisor and what s/he could have done to enhance your experience;<br>5. Use the designation used in the question (Associate, Brand Ambassador, Advisor, etc.); Do n

### Extract Questions

In [2]:
#2
# Extract all questions (ID and text)
#Creates a dictionary:
#For each question (QuestionID), saves its text.
#For each answer (AnswerSetID + Position), saves answer text.
questions = {}
for el in survey_structure[0]["data"]["SurveyFormElements"]:
    f = el["fields"]
    if f["ElementTypeID"].strip() == "Q" and f["ElementID"]:
        questions[f["ElementID"]] = f["Text"]

answers = {}
for ans in survey_structure[0]["data"]["SurveyFormQuestionAnswers"]:
    f = ans["fields"]
    qid = f["QuestionID"]
    pos = f["Position"]
    answers[(qid, pos)] = f["Text"]

print("Sample Questions:")
for k, v in list(questions.items())[:3]:
    print(f"{k}: {v}")
print("\nSample Answers:")
for k, v in list(answers.items())[:5]:
    print(f"{k}: {v}")


Sample Questions:
397320: <p>Please note that the questionnaire is not as long as it seems - multiple questions will be hidden based on your scenario.</p><p>Please read the guidelines below before filling in the questionnaire. <br><br>1. Read the questions carefully before answering to understand what exactly it is asking you;<br>2. Your feedback should describe in detail how you felt during the visit/interaction with the Advisor;<br>3. The Advisor's performance is the focus of this questionnaire, not prices, products, or store location;<br>4. Please share constructive feedback on what can be controlled by the Advisor and what s/he could have done to enhance your experience;<br>5. Use the designation used in the question (Associate, Brand Ambassador, Advisor, etc.); Do not use her/his name unless it is precisely asked. Use the correct gender when referring to the Advisor in the comments.<br>6. For each question, the selected answer and comment should be consistent;<br>7. Use the past t

### Load Data

In [3]:
#3
#Loads the first response JSON, drills into its data block

with open("data_raw/SurveyInstanceData_28316.json", "r", encoding="utf-8") as f:
    responses = json.load(f)

# Dig into the actual data structure (should be in responses[0]["data"])
data = responses[0].get('data', None)
if data is not None:
    print("Keys in data:", list(data.keys()))
    for k in data.keys():
        items = data[k]
        print(f"\nKey: {k} - #items: {len(items) if hasattr(items, '__len__') else 'n/a'}")
        if isinstance(items, list):
            print("Sample:", items[:2])
else:
    print("No 'data' key found in first response.")




Keys in data: ['QuestionComments', 'QuestionAnswers', 'CustomProperties']

Key: QuestionComments - #items: 10000
Sample: [{'uuid': 'ed488708-5efa-4254-9c56-cd3db576beff', 'fields': {'SurveyInstanceID': 3009453, 'QuestionID': 405576, 'QuestionObjectName': None, 'Comment': '-'}}, {'uuid': '8cfd648b-e436-4012-91c1-c56eda4c6105', 'fields': {'SurveyInstanceID': 3009554, 'QuestionID': 405979, 'QuestionObjectName': None, 'Comment': '-'}}]

Key: QuestionAnswers - #items: 228761
Sample: [{'uuid': '5aaba874-6c2c-4758-b43f-a603293473da', 'fields': {'SurveyInstanceID': 3006383, 'QuestionID': 405552, 'QuestionObjectName': None, 'AnswerPos': 1, 'AnswerObjectName': None, 'AnswerIsOtherComment': None}}, {'uuid': 'b1756232-91b0-4c38-a6c5-de333af5c18f', 'fields': {'SurveyInstanceID': 3006383, 'QuestionID': 405552, 'QuestionObjectName': None, 'AnswerPos': 2, 'AnswerObjectName': None, 'AnswerIsOtherComment': None}}]

Key: CustomProperties - #items: 14378
Sample: [{'uuid': 'b6aa02d8-a275-472e-9f34-047e9ce1

### Find Questions

In [4]:
#4
#Builds lookup tables to decide whether each QuestionID allows multiple selections 
# by marrying QuestionAnswerSetProperties with the answer list
answer_sets = survey_structure[0]['data']['QuestionAnswerSetProperties']

answer_set_id_to_multiple = {
    a['fields']['AnswerSetID']: a['fields'].get('IsMultipleSelection', False)
    for a in answer_sets
}
# Build question_answers: maps QuestionID to list of answer dicts (from survey_structure)
question_answers = {}
for a in survey_structure[0]["data"]["SurveyFormQuestionAnswers"]:
    qid = a["fields"]["QuestionID"]
    question_answers.setdefault(qid, []).append(a["fields"])

def is_multi(qid):
    ans = question_answers.get(qid, [])
    if ans:
        as_etid = ans[0]['AnswerSetID']
        return answer_set_id_to_multiple.get(as_etid, False)
    return False

# Show if a few questions are multi-select
sample = []
for qid, qtext in list(questions.items())[:10]:
    sample.append({
        'ElementID': qid,
        'Text': (qtext or '')[:40],
        'IsMulti': is_multi(qid)
    })
pd.DataFrame(sample)



,ElementID,Text,IsMulti
0,397320,<p>Please note that the questionnaire is,False
1,391189,What was your scenario?,False
2,391222,"Based on your experience on the website,",False
3,383785,Please select the main reasons why the w,True
4,368206,Please select the main reasons why you d,True
5,358573,Did you book an appointment on the websi,False
6,383198,Based on your appointment booking experi,False
7,391244,Please select the main reasons why you f,True
8,391245,Please select the main reasons why you d,True
9,383193,Were you contacted by an Advisor after y,True


### Build a minimal "pivot" DataFrame

In [5]:
#5a

# Convert QuestionAnswers to a DataFrame

qa = pd.DataFrame(data["QuestionAnswers"])
# Expand 'fields' dict into columns
qa_fields = qa['fields'].apply(pd.Series)

# Preview columns and data
print("QA fields columns:", qa_fields.columns.tolist())
print(qa_fields.head())

# Just check how many unique SurveyInstanceID, QuestionID, and AnswerPos we have
print("Unique instances:", qa_fields['SurveyInstanceID'].nunique())
print("Unique questions:", qa_fields['QuestionID'].nunique())
print("Sample AnswerPos:", qa_fields['AnswerPos'].unique()[:5])


QA fields columns: ['SurveyInstanceID', 'QuestionID', 'QuestionObjectName', 'AnswerPos', 'AnswerObjectName', 'AnswerIsOtherComment']
   SurveyInstanceID  QuestionID  QuestionObjectName  AnswerPos  \
0         3006383.0    405552.0                 NaN        1.0   
1         3006383.0    405552.0                 NaN        2.0   
2         3006383.0    405552.0                 NaN        3.0   
3         3006383.0    405552.0                 NaN        4.0   
4         3006383.0    405553.0                 NaN        1.0   

   AnswerObjectName  AnswerIsOtherComment  
0               NaN                   NaN  
1               NaN                   NaN  
2               NaN                   NaN  
3               NaN                   NaN  
4               NaN                   NaN  
Unique instances: 2054
Unique questions: 116
Sample AnswerPos: [1. 2. 3. 4. 5.]


In [6]:
# 5b
# Create a column for pivot: QuestionID_AnswerPos
qa_fields['col'] = qa_fields['QuestionID'].astype(int).astype(str) + "_" + qa_fields['AnswerPos'].astype(int).astype(str)
qa_fields['value'] = 1

# Pivot to wide (one-hot) format
pivot = qa_fields.pivot_table(index="SurveyInstanceID", columns="col", values="value", fill_value=0, aggfunc="max")

print("Pivot shape:", pivot.shape)
display(pivot.head())


Pivot shape: (2054, 696)


col,358573_1,358573_2,360187_1,360187_12,360187_14,360187_15,360187_16,360187_17,360187_18,360187_19,...,406171_2,406171_3,406171_4,406171_5,406171_6,406171_7,406171_8,406181_1,406181_2,406181_3
SurveyInstanceID,,,,,,,,,,,,,,,,,,,,,
3005492.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005493.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005494.0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3005495.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005496.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Collapse 2 Answer Questions

In [7]:
# 6
# Collapse 2-option questions to a single column 

# Helper to recognise a "no/negative" answer label
def looks_like_no(txt):
    return str(txt).strip().lower() in {"no", "none", "n/a", "na", "false", "0", "nope"}

cols_to_drop = []

# Group current wide columns by QuestionID (prefix before the underscore)
from collections import defaultdict
qid_to_cols = defaultdict(list)
for col in pivot.columns:
    qid, pos = col.split("_", 1)
    qid_to_cols[qid].append((int(pos), col))

for qid, pos_col_list in qid_to_cols.items():
    # Only care about questions that produced exactly two columns
    if len(pos_col_list) == 2:
        pos_col_list.sort()  # sort by AnswerPos for determinism
        (pos1, col1), (pos2, col2) = pos_col_list

        # Try to read the answer labels from the dict you built earlier
        label1 = answers.get((int(qid), pos1), "")
        label2 = answers.get((int(qid), pos2), "")

        # Prefer to drop the column whose label looks like "No"
        if looks_like_no(label1):
            drop_col = col1
        elif looks_like_no(label2):
            drop_col = col2
        else:
            # Fallback: drop the one that is "mostly zeros" (lower mean of 1s)
            mean1 = pivot[col1].mean()
            mean2 = pivot[col2].mean()
            drop_col = col1 if mean1 < mean2 else col2

        cols_to_drop.append(drop_col)

# Actually drop them and keep going with the same variable name (`pivot`)
pivot = pivot.drop(columns=cols_to_drop)

print(f"Dropped {len(cols_to_drop)} columns from binary questions.")
pivot.head()


Dropped 10 columns from binary questions.


col,358573_1,360187_1,360187_12,360187_14,360187_15,360187_16,360187_17,360187_18,360187_19,360187_2,...,406171_2,406171_3,406171_4,406171_5,406171_6,406171_7,406171_8,406181_1,406181_2,406181_3
SurveyInstanceID,,,,,,,,,,,,,,,,,,,,,
3005492.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005493.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005494.0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3005495.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3005496.0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Dataframe Schema

In [8]:
#7
#generates a full list of readable DataFrame column names, using the survey structure


# Setup: build a lookup for which AnswerSetIDs are multi-select
answer_sets = survey_structure[0]["data"]["QuestionAnswerSetProperties"]
answer_set_id_to_multiple = {
    a['fields']['AnswerSetID']: a['fields'].get('IsMultipleSelection', False)
    for a in answer_sets
}

# Build: map each QuestionID to its AnswerSetID (from structure)
qid_to_asid = {}
for ans in survey_structure[0]["data"]["SurveyFormQuestionAnswers"]:
    f = ans["fields"]
    qid = f["QuestionID"]
    asid = f["AnswerSetID"]
    qid_to_asid[qid] = asid

def is_multi(qid):
    asid = qid_to_asid.get(qid)
    return answer_set_id_to_multiple.get(asid, False)

# Now generate readable columns for the pivot table
columns = []
for qid in questions:
    ans = [a for (qid2, pos), a in answers.items() if qid2 == qid]
    if ans and is_multi(qid):
        for pos in range(1, len(ans)+1):
            label = answers.get((qid, pos), f"Ans{pos}")
            label = str(label).replace('\n', ' ').replace('\r', ' ')
            columns.append(f"{qid}_{pos}: {label}")
    else:
        columns.append(f"{qid}: {questions[qid]}")
        
print(f"Total columns: {len(columns)}")
print(columns[:15])



Total columns: 536
["397320: <p>Please note that the questionnaire is not as long as it seems - multiple questions will be hidden based on your scenario.</p><p>Please read the guidelines below before filling in the questionnaire.\xa0<br><br>1. Read the questions carefully before answering to understand what exactly it is asking you;<br>2. Your feedback should describe in detail how you felt during the visit/interaction with the Advisor;<br>3. The Advisor's performance is the focus of this questionnaire, not prices, products, or store location;<br>4. Please share constructive feedback on what can be controlled by the Advisor and what s/he could have done to enhance your experience;<br>5. Use the designation used in the question (Associate, Brand Ambassador, Advisor, etc.); Do not use her/his name unless it is precisely asked. Use the correct gender when referring to the Advisor in the comments.<br>6. For each question, the selected answer and comment should be consistent;<br>7. Use the 

### Produce DataFrame

In [9]:
# 8 ── Keep column names strictly as IDs  ──────────────────────────────────────
# (e.g. "405982_3", without the ": All questions were clear …" suffix)

# Build a mapping that strips anything following the first colon, if present.
pretty_map = {col: col.split(":", 1)[0] for col in pivot.columns}

# Apply mapping and make SurveyInstanceID a normal column
pivot_pretty = (
    pivot
      .rename(columns=pretty_map)
      .reset_index()
)

# Quick preview
display(pivot_pretty.head())


col,SurveyInstanceID,358573_1,360187_1,360187_12,360187_14,360187_15,360187_16,360187_17,360187_18,360187_19,...,406171_2,406171_3,406171_4,406171_5,406171_6,406171_7,406171_8,406181_1,406181_2,406181_3
0,3005492.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3005493.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3005494.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,3005495.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3005496.0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### Adding Total 1s

In [10]:
# 8A ── Convert free-text “Other” answers → binary flags
import re

source_df = pivot_pretty          # this dataframe exists already

# 1) columns that look like free-text “Other”
other_text_cols = [
    c for c in source_df.columns
    if re.search(r'other', c, flags=re.I) and source_df[c].dtype == "object"
]

# 2) create <QuestionID>_other flags (1 = respondent typed something)
for col in other_text_cols:
    qid_match = re.match(r"(\d+)", col)      # grab the numeric QuestionID
    if not qid_match:
        continue
    qid = qid_match.group(1)
    pivot_pretty[f"{qid}_other"] = source_df[col].notna().astype("Int8")

# 3) drop the raw free-text columns
pivot_pretty = pivot_pretty.drop(columns=other_text_cols, errors="ignore")


In [11]:
# # 8B ── Close any numeric gaps in answer-position columns
# #       Example: 383785_1 … 383785_7 383785_9  →  383785_1 … 383785_7 383785_8
# #       The free-text “Other” checkbox (if present as *_other or *_11) becomes
# #       the next index after the last numeric option.

# import re
# from collections import defaultdict

# # 1) collect numeric positions and any “Other” column per question
# qid_numeric = defaultdict(list)   # {qid: [pos, pos, …]}
# qid_other   = {}                  # {qid: 'col_name_marking_other'}

# for col in pivot_pretty.columns:
#     m_num  = re.match(r'^(\d+)_(\d+)$', col)        # e.g. 383785_9
#     m_oth  = re.match(r'^(\d+)_other$', col)        # e.g. 383785_other
#     m_11   = re.match(r'^(\d+)_11$',   col)         # e.g. 383785_11  (often Other)

#     if m_num:
#         qid, pos = m_num.groups()
#         qid_numeric[qid].append(int(pos))
#     elif m_oth:
#         qid_other[m_oth.group(1)] = col
#     elif m_11:                                      # treat *_11 as “Other”
#         qid_other[m_11.group(1)] = col

# # 2) build a rename-map so numeric indices become consecutive 1…N
# rename_map = {}

# for qid, positions in qid_numeric.items():
#     positions_sorted = sorted(set(positions))
#     for new_idx, old_idx in enumerate(positions_sorted, start=1):
#         old_name = f"{qid}_{old_idx}"
#         new_name = f"{qid}_{new_idx}"
#         if old_name != new_name:
#             rename_map[old_name] = new_name

#     # handle “Other” → next sequential index
#     if qid in qid_other:
#         other_old = qid_other[qid]
#         next_idx  = len(positions_sorted) + 1
#         new_name  = f"{qid}_{next_idx}"
#         # if collision (very unlikely) keep incrementing
#         while new_name in pivot_pretty.columns:
#             next_idx += 1
#             new_name = f"{qid}_{next_idx}"
#         rename_map[other_old] = new_name

# # 3) apply renaming
# pivot_pretty.rename(columns=rename_map, inplace=True)


In [12]:
# 9
# Add a bottom row with % of 1s per column

# Work on a copy so you can still inspect the original if needed
df_out = pivot_pretty.copy()

# Columns that should be checked for 1s (exclude IDs or text cols)
non_binary_cols = ['SurveyInstanceID']  # add more here if needed
target_cols = [c for c in df_out.columns if c not in non_binary_cols]

# Compute % of 1s
pct_ones = (df_out[target_cols].eq(1).sum() / len(df_out)) * 100

# Build the summary row
summary_row = pd.Series(index=df_out.columns, dtype='object')
summary_row[target_cols] = pct_ones.round(2)
summary_row[non_binary_cols] = 'Pct_1s'   # label cell(s) so you can spot the row

# Append to the bottom
df_out = pd.concat([df_out, summary_row.to_frame().T], ignore_index=True)

# If you want downstream cells (e.g., export) to use it:
pivot_pretty = df_out


### Merging Data Low Total

In [13]:
# 10 
# Collapse rare answers (<5%) into “Other” buckets, but ignore Likert cols

import re
from collections import defaultdict
import pandas as pd

# ── 0) List of all 16 Likert base IDs to skip ───────────────────────────────
LIKERT_BASE_IDS = {
    "391222","383198","383199","391201","391213",
    "380319","380514","405548","383195","380437",
    "397317","380448","380442","405583","405568","405978"
}

# ── 1) Remove summary row we added earlier ─────────────────────────────────
summary_mask = pivot_pretty['SurveyInstanceID'].eq('Pct_1s')
df = pivot_pretty.loc[~summary_mask].copy()

# ── 2) Identify candidate columns to examine (exclude ID column) ────────────
non_binary_cols = ['SurveyInstanceID']
target_cols     = [c for c in df.columns if c not in non_binary_cols]

# ── 3) Compute % of 1s per column; find columns under 5% (and non-Likert) ──
pct_ones = df[target_cols].eq(1).mean() * 100

def split_col(col):
    head = col.split(':', 1)[0]
    if '_' in head:
        qid, pos = head.split('_', 1)
        try:
            return qid.strip(), int(pos)
        except ValueError:
            return qid.strip(), None
    return head.strip(), None

def get_qid(col):
    return split_col(col)[0]

# Only collapse non-Likert columns whose positive rate is below 5%
rare_cols = [
    c for c, pct in pct_ones.items()
    if pct < 5 and get_qid(c) not in LIKERT_BASE_IDS
]

# ── 4) Group rare cols by QuestionID ────────────────────────────────────────
qid_to_rare = defaultdict(list)
for c in rare_cols:
    qid_to_rare[get_qid(c)].append(c)

# ── 5) Build new “Other” bucket columns ────────────────────────────────────
new_cols_data = {}
merge_log    = {}
merged_count = 0

for qid, cols in qid_to_rare.items():
    # find existing answer positions for this question
    existing_pos = [
        split_col(c)[1]
        for c in df.columns
        if split_col(c)[0] == qid and split_col(c)[1] is not None
    ]
    next_pos = (max(existing_pos) + 1) if existing_pos else 1

    # ensure unique new column name
    new_col = f"{qid}_{next_pos}"
    while new_col in df.columns or new_col in new_cols_data:
        next_pos += 1
        new_col = f"{qid}_{next_pos}"

    # collapse: mark 1 if any of the rare columns was chosen
    new_cols_data[new_col] = df[cols].any(axis=1).astype(int)
    merge_log[new_col]    = cols
    merged_count += len(cols)

    # if you maintain an answers dict for lookups, label it “Other”
    if 'answers' in globals():
        answers[(qid, next_pos)] = "Other"

# ── 6) Apply changes: drop old rare cols, add new buckets ───────────────────
others_df = pd.DataFrame(new_cols_data, index=df.index)
df = pd.concat([df.drop(columns=rare_cols, errors='ignore'), others_df], axis=1)

# ── 7) Rebuild the summary row and reattach it ─────────────────────────────
new_target_cols  = [c for c in df.columns if c not in non_binary_cols]
new_pct_ones     = (df[new_target_cols].eq(1).mean() * 100).round(2)

summary_row = pd.Series(index=df.columns, dtype='object')
summary_row[new_target_cols] = new_pct_ones
summary_row[non_binary_cols] = 'Pct_1s'

pivot_pretty = pd.concat([df, summary_row.to_frame().T], ignore_index=True)

print(f"{merged_count} columns were merged into new low-frequency buckets.")
print("Example new bucket(s):", list(merge_log.keys())[:5])


316 columns were merged into new low-frequency buckets.
Example new bucket(s): ['360187_24', '360190_4', '368206_9', '369184_6', '369188_6']


##### Data Summary on Others

In [14]:
# 11a
#  Summary of "% Other" buckets (robust, no .str.split)

summary_mask = pivot_pretty['SurveyInstanceID'].eq('Pct_1s')
df_only      = pivot_pretty.loc[~summary_mask]

# All "Other" columns created in #10a
other_cols = list(merge_log.keys())

n_rows = len(df_only)

# Counts & % for each "_Other" bucket column
counts = df_only[other_cols].sum()

other_summary = counts.to_frame('count_1s')
other_summary['pct_1s'] = (other_summary['count_1s'] / n_rows * 100).round(2)

def parse_qid_pos(col):
    head = col.split(':', 1)[0]          # strip pretty text if present
    parts = head.split('_', 1)
    qid = parts[0]
    pos = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else None
    return qid, pos

qid_pos = [parse_qid_pos(c) for c in other_summary.index]
other_summary['qid'] = [q for q, p in qid_pos]
other_summary['pos'] = [p for q, p in qid_pos]

# Question text and label
other_summary['question_text'] = other_summary['qid'].map(
    lambda q: questions.get(int(q), questions.get(q, '[Question not found]'))
)
other_summary['answer_text']   = 'Other'

# What was merged
other_summary['merged_from'] = other_summary.index.map(lambda c: merge_log.get(c, []))
other_summary['merged_n']    = other_summary['merged_from'].str.len()

# % "Other" per question (combine if multiple buckets)
by_question = (
    other_summary.groupby('qid')['count_1s'].sum() / n_rows * 100
).round(2).rename('pct_other')

# Overall % of respondents who hit ANY "Other"
overall_other_pct = (df_only[other_cols].any(axis=1).mean() * 100).round(2)

print(f'Overall pct of respondents selecting any "Other": {overall_other_pct}%')
print('\nFirst 5 rows of other_summary:')
print(other_summary.head())
print('\nFirst 5 rows of by_question:')
print(by_question.head())


Overall pct of respondents selecting any "Other": 83.93%

First 5 rows of other_summary:
          count_1s     pct_1s     qid  pos  \
360187_24      300  14.605648  360187   24   
360190_4        52   2.531646  360190    4   
368206_9        63   3.067186  368206    9   
369184_6        42   2.044791  369184    6   
369188_6        94   4.576436  369188    6   

                                               question_text answer_text  \
360187_24                           Which area are you from?       Other   
360190_4                                          Are you a:       Other   
368206_9   Please select the main reasons why you did not...       Other   
369184_6   Based on your journey across these different t...       Other   
369188_6   Based on your experience, to which extent woul...       Other   

                                                 merged_from  merged_n  
360187_24  [360187_1, 360187_12, 360187_14, 360187_15, 36...        19  
360190_4                       

In [15]:
# 11b
# Save the newly calculated summaries to /data_processed

from pathlib import Path

out_dir = Path("data_processed")
out_dir.mkdir(parents=True, exist_ok=True)

# Tidy up frames for writing
other_summary_out = other_summary.reset_index().rename(columns={'index': 'col_name'})
by_question_out   = by_question.reset_index().rename(columns={'qid': 'QuestionID'})

merge_log_out = pd.DataFrame(
    [(k, v) for k, v in merge_log.items()],
    columns=['new_col', 'merged_from']
)

# Option A: separate workbook just for summaries
with pd.ExcelWriter(out_dir / "other_summaries.xlsx", engine="openpyxl") as w:
    other_summary_out.to_excel(w, sheet_name="other_cols", index=False)
    by_question_out.to_excel(w, sheet_name="pct_other_by_qid", index=False)
    merge_log_out.to_excel(w, sheet_name="merge_log", index=False)

# Option B (commented): append as sheets to your existing export
# with pd.ExcelWriter(out_dir / "survey_dataframe.xlsx",
#                     engine="openpyxl", mode="a", if_sheet_exists="replace") as w:
#     other_summary_out.to_excel(w, sheet_name="other_cols", index=False)
#     by_question_out.to_excel(w, sheet_name="pct_other_by_qid", index=False)
#     merge_log_out.to_excel(w, sheet_name="merge_log", index=False)

print("Saved summaries to data_processed/other_summaries.xlsx")


Saved summaries to data_processed/other_summaries.xlsx


### Load Data

In [16]:
# 12
# Load instance data and drill down to actual list of answers

with open('data_raw/SurveyInstanceData_28316.json', 'r', encoding='utf-8') as f:
    instance_data = json.load(f)

# Navigate: list -> dict ('data') -> dict (with 'QuestionAnswers') -> the giant list
answers_list = instance_data[0]['data']['QuestionAnswers']

print("Type of answers_list:", type(answers_list))
print("Number of answer records:", len(answers_list))
print("Keys in first answer record:", list(answers_list[0].keys()))
print("Sample of first answer record:\n", answers_list[0])


Type of answers_list: <class 'list'>
Number of answer records: 228761
Keys in first answer record: ['uuid', 'fields']
Sample of first answer record:
 {'uuid': '5aaba874-6c2c-4758-b43f-a603293473da', 'fields': {'SurveyInstanceID': 3006383, 'QuestionID': 405552, 'QuestionObjectName': None, 'AnswerPos': 1, 'AnswerObjectName': None, 'AnswerIsOtherComment': None}}


### Printing Dataframe

In [17]:
#13

pivot_pretty.to_excel("data_processed/survey_dataframe.xlsx", index=False)
print("Output written to survey_dataframe.xlsx")


Output written to survey_dataframe.xlsx


### Mapping Names

In [18]:
#14
# Creating a mapping for all the column names on a second sheet

import re
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook

# --- build mapping dataframe -----------------------------------------------

rows = []
for col in pivot_pretty.columns:
    if col == "SurveyInstanceID":
        continue                      # skip the identifier column

    # Strip any pretty label suffix after ":"
    head = col.split(":", 1)[0]

    # Parse QuestionID and AnswerPos if present
    if "_" in head:
        qid, pos_str = head.split("_", 1)
        try:
            pos = int(pos_str)
        except ValueError:
            pos = None
    else:
        qid, pos = head, None

    # Retrieve text
    q_text = questions.get(int(qid), questions.get(qid, "[Question not found]"))

    # answer label lookup tries both str & int keys
    a_label = (
        answers.get((qid, pos)) or
        answers.get((int(qid), pos))
    )

    # if still missing and this is one of our "Other" buckets
    if a_label is None and 'merge_log' in globals() and head in merge_log:
        a_label = "Other"

    rows.append({
        "col_name":      col,
        "QuestionID":    qid,
        "AnswerPos":     pos,
        "QuestionText":  q_text,
        "AnswerLabel":   a_label,
    })

mapping_df = pd.DataFrame(rows)

# --- write to Excel ---------------------------------------------------------

out_path = Path("data_processed/survey_dataframe.xlsx")
with pd.ExcelWriter(out_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    mapping_df.to_excel(writer, sheet_name="mapping", index=False)

print(f"Mapping sheet written to {out_path} (sheet name: 'mapping')")


Mapping sheet written to data_processed\survey_dataframe.xlsx (sheet name: 'mapping')


### Filter Social Media Responses

In [19]:
# #15a
# # Filtering the specific social media question to a sperate dataframe to analyse closer


# candidates = [
#     (qid, txt) for qid, txt in questions.items()
#     if "social media" in txt.lower()
# ]

# print(f"Found {len(candidates)} questions containing 'social media':\n")
# for qid, txt in candidates:
#     print(f"{qid}: {txt[:120]}{'...' if len(txt) > 120 else ''}")

In [20]:
# # 15b
# # isolate the columns & build the dataframe


# social_qids = {str(qid) for qid, _ in candidates}   # use ALL discovered qids
# print("Using QuestionIDs:", social_qids)

# def belongs(col):
#     head = col.split(":", 1)[0]
#     qid  = head.split("_", 1)[0]
#     return qid in social_qids

# sm_cols  = ["SurveyInstanceID"] + [c for c in pivot_pretty.columns if belongs(c)]
# social_df = pivot_pretty[sm_cols].copy()

# print("social_df shape:", social_df.shape)
# social_df.head()

In [21]:
# # 15c
# # Write to excell, analyse data

# from pathlib import Path
# import pandas as pd

# data_only = social_df[social_df['SurveyInstanceID'] != 'Pct_1s']
# n_rows    = len(data_only)

# rows = []
# for col in social_df.columns:
#     if col == "SurveyInstanceID":
#         continue
#     head = col.split(":", 1)[0]
#     parts = head.split("_", 1)
#     qid = parts[0]
#     pos = int(parts[1]) if len(parts) > 1 and parts[1].isdigit() else None

#     count = int(data_only[col].sum())
#     pct   = round(count / n_rows * 100, 2)

#     q_txt = questions.get(int(qid), questions.get(qid, "[Question not found]"))
#     a_txt = (
#         answers.get((qid, pos)) or
#         answers.get((int(qid), pos)) or
#         ("Other" if head in merge_log else "[Answer label not found]")
#     )

#     rows.append({
#         "QuestionID":  qid,
#         "AnswerPos":   pos,
#         "Question":    q_txt,
#         "AnswerLabel": a_txt,
#         "count_1s":    count,
#         "pct_1s":      pct,
#         "col_name":    col,
#     })

# analysis_df = pd.DataFrame(rows)

# out_path = Path("data_processed/social_media_dataframe.xlsx")
# with pd.ExcelWriter(out_path, engine="openpyxl") as w:
#     social_df.to_excel(w, sheet_name="data",     index=False)
#     analysis_df.to_excel(w, sheet_name="analysis", index=False)

# print(f"Saved Social‑Media workbook ➜ {out_path}")

### Scan for Duplicates

In [22]:
# # 18  ── Data-quality scan
# from collections import Counter          # ← add this

# report = {}

# # 1. duplicate SurveyInstanceID rows
# dup_ids = pivot_pretty['SurveyInstanceID'].value_counts()
# report['duplicate_ids'] = dup_ids[dup_ids > 1].index.tolist()

# # 2. columns that are entirely zero (no one selected them)
# binary_cols = [c for c in pivot_pretty.columns if c != 'SurveyInstanceID']
# all_zero    = [c for c in binary_cols if pivot_pretty[c].dropna().sum() == 0]
# report['all_zero_cols'] = all_zero

# # 3. completely blank rows (no answers ticked at all)
# blank_rows = (pivot_pretty[binary_cols].sum(axis=1) == 0).sum()
# report['blank_rows'] = int(blank_rows)

# # 4. single-choice questions with >1 option ticked in a row
# violations = Counter()
# for qid in {c.split('_', 1)[0] for c in binary_cols}:
#     q_cols = [c for c in binary_cols if c.startswith(f'{qid}_')]
#     if len(q_cols) <= 1:          # multi-select or scalar — ignore
#         continue
#     too_many = (pivot_pretty[q_cols].sum(axis=1) > 1).sum()
#     if too_many:
#         violations[qid] = int(too_many)
# report['single_choice_violations'] = dict(violations)

# # Show the report
# print("=== Data-quality report ===")
# for k, v in report.items():
#     print(f"{k}: {v if v else 'OK'}")


In [23]:
# # 19 — Response‑rate heat‑map  ( % of respondents selecting each answer )

# import matplotlib.pyplot as plt
# import numpy as np

# # ── 1.  Work on real‑response rows only ────────────────────────────────────────
# data_df = pivot_pretty[pivot_pretty['SurveyInstanceID'] != 'Pct_1s']

# binary_cols = [c for c in data_df.columns if c != "SurveyInstanceID"]

# def split_col(col):
#     """"""Return (qid, pos_int) from a column name like '360187_3: …'.""""""""
#     head = col.split(":", 1)[0]          # strip pretty suffix if present
#     if "_" in head:
#         qid, pos = head.split("_", 1)
#         try:
#             pos = int(pos)
#         except ValueError:
#             pos = 1                      # safety fallback
#     else:
#         qid, pos = head, 1
#     return qid, pos

# records = []
# for col in binary_cols:
#     qid, pos = split_col(col)
#     pct = round(data_df[col].mean() * 100, 2)   # % of respondents who marked 1
#     records.append((qid, pos, pct))

# heat_df = (
#     pd.DataFrame(records, columns=["qid", "pos", "pct"])
#       .pivot(index="qid", columns="pos", values="pct")
#       .sort_index()
#       .fillna(0)
# )

# # ── 2.  Plot heat‑map (single plot, no custom colors) ─────────────────────────
# plt.figure(figsize=(12, max(4, heat_df.shape[0] * 0.25)))
# plt.imshow(heat_df.values, aspect="auto")
# plt.colorbar(label="% respondents")
# plt.xticks(np.arange(len(heat_df.columns)), heat_df.columns)
# plt.yticks(np.arange(len(heat_df.index)), heat_df.index)
# plt.title("Response‑rate heat‑map")
# plt.xlabel("Answer position")
# plt.ylabel("QuestionID")
# plt.tight_layout()
# plt.show()


In [24]:
# # === QA CHECK CELL (place at the end) =========================================
# # Choose a QuestionID and verify the rare-answer merging.
# # - Builds a summary table for that question with count of 1s and % of 1s
# # - Shows which columns are auto-created "Other" buckets
# # - Lists all datapoints (SurveyInstanceID) for columns under 5%
# # - Checks that listed datapoints match the summarized counts
# #
# # Set qid_selected to a specific ID (e.g., "360187") or leave as None to auto-pick.

# THRESHOLD_PCT = 5.0        # "under 5%" threshold
# qid_selected = None        # e.g. "360187" or 360187

# def _extract_head(col: str) -> str:
#     # part before any pretty suffix like ': Label text'
#     return col.split(":", 1)[0]

# def _split_col(head: str):
#     # returns (qid_str, pos_int) ; pos=1 for single-choice style heads
#     if "_" in head:
#         qid_str, pos_str = head.split("_", 1)
#         pos_str = re.sub(r"\D+", "", pos_str) or "1"
#         try:
#             pos = int(pos_str)
#         except ValueError:
#             pos = 1
#         return qid_str, pos
#     return head, 1

# def _label_for(qid_str: str, pos: int):
#     # Fetch human-readable texts from `questions` and `answers` where available
#     q_text = "[Question text not found]"
#     a_label = None

#     # Question text
#     if "questions" in globals():
#         try:
#             qid_int = int(qid_str)
#         except ValueError:
#             qid_int = None
#         if qid_int is not None:
#             q_text = questions.get(qid_int, questions.get(qid_str, q_text))
#         else:
#             q_text = questions.get(qid_str, q_text)

#     # Answer label
#     if "answers" in globals():
#         try:
#             qid_int = int(qid_str)
#         except ValueError:
#             qid_int = None
#         if qid_int is not None:
#             a_label = answers.get((qid_int, pos), answers.get((qid_str, pos)))
#         else:
#             a_label = answers.get((qid_str, pos))

#     # If still missing and this head was created as an "Other" bucket
#     if a_label is None and "merge_log" in globals():
#         head = f"{qid_str}_{pos}"
#         if head in merge_log:
#             a_label = "Other"

#     if a_label is None:
#         a_label = "[Answer label not found]"
#     return q_text, a_label

# # ---- guards & base frame -----------------------------------------------------
# if "pivot_pretty" not in globals():
#     raise RuntimeError("pivot_pretty is not defined. Run the cells that build it first.")

# # Remove the summary row if present
# if "SurveyInstanceID" in pivot_pretty.columns:
#     data_df = pivot_pretty.loc[~pivot_pretty["SurveyInstanceID"].astype(str).eq("Pct_1s")].copy()
# else:
#     data_df = pivot_pretty.copy()

# n_rows = len(data_df)
# non_binary_cols = ["SurveyInstanceID"] if "SurveyInstanceID" in data_df.columns else []
# data_cols = [c for c in data_df.columns if c not in non_binary_cols]

# # Build map of qid -> count of columns to help pick a default
# qid_counts = {}
# for col in data_cols:
#     head = _extract_head(col)
#     qid_str, _ = _split_col(head)
#     qid_counts[qid_str] = qid_counts.get(qid_str, 0) + 1

# if qid_selected is None:
#     if not qid_counts:
#         raise RuntimeError("No data columns found to infer QuestionIDs.")
#     qid_selected = max(qid_counts.items(), key=lambda kv: kv[1])[0]
# qid_selected = str(qid_selected)

# # Collect columns for the chosen question
# rows_meta = []
# for col in data_cols:
#     head = _extract_head(col)
#     qid_str, pos = _split_col(head)
#     if qid_str == qid_selected:
#         is_other = ("merge_log" in globals()) and (head in merge_log)
#         rows_meta.append((col, head, pos, is_other))

# if not rows_meta:
#     raise RuntimeError(f"No columns found for QuestionID {qid_selected}. "
#                        f"Available QuestionIDs: {sorted(qid_counts.keys())[:20]}{' …' if len(qid_counts)>20 else ''}")

# # Order: originals first, then new "Other" columns
# rows_meta.sort(key=lambda t: (t[3], t[2]))  # (is_other, AnswerPos)

# # Build summary table
# summary_rows = []
# for col, head, pos, is_other in rows_meta:
#     q_text, a_label = _label_for(qid_selected, pos)
#     count_1s = int(data_df[col].astype(int).sum())
#     pct_1s = round((count_1s / n_rows) * 100, 2) if n_rows else 0.0
#     summary_rows.append({
#         "QuestionID": qid_selected,
#         "QuestionText": q_text,
#         "col_name": col,
#         "head": head,
#         "AnswerPos": pos,
#         "is_other": bool(is_other),
#         "AnswerLabel": a_label,
#         "count_1s": count_1s,
#         "pct_1s": pct_1s,
#         "n_rows": n_rows,
#     })

# summary_q = pd.DataFrame(summary_rows)

# print(f"Chosen QuestionID: {qid_selected}")
# if "questions" in globals():
#     try:
#         qid_int = int(qid_selected)
#     except ValueError:
#         qid_int = None
#     q_text_print = questions.get(qid_int, questions.get(qid_selected)) if qid_int is not None else questions.get(qid_selected)
#     if q_text_print:
#         print("Question text:", q_text_print)

# display(summary_q)

# # Columns under the threshold
# low_q = summary_q[summary_q["pct_1s"] < THRESHOLD_PCT].copy()
# print(f"\nColumns under {THRESHOLD_PCT}%: {len(low_q)}")
# display(low_q[["col_name", "AnswerPos", "is_other", "AnswerLabel", "count_1s", "pct_1s"]])

# # Show datapoints (SurveyInstanceID values) for each low-frequency column
# records = []
# for _, r in low_q.iterrows():
#     col = r["col_name"]
#     if "SurveyInstanceID" in data_df.columns:
#         ids = data_df.loc[data_df[col] == 1, "SurveyInstanceID"].astype(str).tolist()
#     else:
#         ids = data_df.index[data_df[col] == 1].astype(str).tolist()

#     records.append({
#         "col_name": col,
#         "observed_count": len(ids),
#         "summary_count_1s": int(r["count_1s"]),
#         "expected_from_pct": round(r["pct_1s"] * r["n_rows"] / 100.0, 2),
#         "SurveyInstanceIDs": ids,
#     })

# low_points = pd.DataFrame(records)
# print("\nDatapoints for columns under threshold:")
# display(low_points)

# # Consistency check
# mismatches = low_points[low_points["observed_count"] != low_points["summary_count_1s"]]
# if mismatches.empty:
#     print("✅ All low-frequency columns match between listed datapoints and summarized counts.")
# else:
#     print("⚠️ Mismatch detected. Review the following rows:")
#     display(mismatches)
# # =============================================================================
